In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
# from keras.datasets import mnist
# from keras.datasets import imdb
# from keras.datasets import reuters
from keras.datasets import boston_housing

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

from keras.utils.np_utils import to_categorical

plt.style.use('ggplot')

%matplotlib inline

Using TensorFlow backend.


In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [3]:
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [5]:
# Normalise data
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [8]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64,
                           activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,
                           activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop',
                  loss='mse',
                  metrics=['mae'])
    return model

In [9]:
# k fold validation

k = 4 
num_val_samples = len(train_data)//4
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold # ',i)
    val_data = train_data[i*num_val_samples:(i+1)*num_val_samples]
    
    val_targets = train_targets[i*num_val_samples:(i+1)*num_val_samples]
    
    partial_train_data = np.concatenate(
    [train_data[:i*num_val_samples],
     train_data[(i+1)*num_val_samples:]],axis=0)
    
    partial_train_targets = np.concatenate(
    [train_targets[:i*num_val_samples],
     train_targets[(i+1)*num_val_samples:]],axis=0)
    
    model = build_model()
    
    model.fit(partial_train_data,
              partial_train_targets,
              epochs=num_epochs,
              batch_size=1,
              verbose=1)
    val_mse,val_mae = model.evaluate(val_data, 
                                     val_targets, 
                                     verbose = 1)
    all_scores.append(val_mae)
    

processing fold #  0
Epoch 1/100
303/303 [==============================] - 2s 7ms/step - loss: 221.4124 - mean_absolute_error: 11.2313
Epoch 2/100
303/303 [==============================] - 1s 2ms/step - loss: 32.1029 - mean_absolute_error: 3.9805
Epoch 3/100
303/303 [==============================] - 1s 2ms/step - loss: 21.1038 - mean_absolute_error: 3.1642
Epoch 4/100
303/303 [==============================] - 1s 2ms/step - loss: 17.1759 - mean_absolute_error: 2.8027
Epoch 5/100
303/303 [==============================] - 1s 2ms/step - loss: 16.1238 - mean_absolute_error: 2.6195
Epoch 6/100
303/303 [==============================] - 1s 2ms/step - loss: 14.3770 - mean_absolute_error: 2.5573
Epoch 7/100
303/303 [==============================] - 1s 3ms/step - loss: 13.8464 - mean_absolute_error: 2.4277
Epoch 8/100
303/303 [==============================] - 1s 3ms/step - loss: 12.7895 - mean_absolute_error: 2.4191
Epoch 9/100
303/303 [==============================] - 1s 2ms/step - loss

303/303 [==============================] - 1s 2ms/step - loss: 5.8678 - mean_absolute_error: 1.6643
Epoch 74/100
303/303 [==============================] - 1s 2ms/step - loss: 5.8344 - mean_absolute_error: 1.5770
Epoch 75/100
303/303 [==============================] - 1s 2ms/step - loss: 5.6974 - mean_absolute_error: 1.5900
Epoch 76/100
303/303 [==============================] - 1s 3ms/step - loss: 5.6422 - mean_absolute_error: 1.5702
Epoch 77/100
303/303 [==============================] - 1s 2ms/step - loss: 5.7444 - mean_absolute_error: 1.6020
Epoch 78/100
303/303 [==============================] - 1s 3ms/step - loss: 5.2883 - mean_absolute_error: 1.5392
Epoch 79/100
303/303 [==============================] - 1s 3ms/step - loss: 5.3258 - mean_absolute_error: 1.5329
Epoch 80/100
303/303 [==============================] - 1s 3ms/step - loss: 5.5784 - mean_absolute_error: 1.5882
Epoch 81/100
303/303 [==============================] - 1s 3ms/step - loss: 5.6858 - mean_absolute_error: 1.5

303/303 [==============================] - 1s 2ms/step - loss: 7.7614 - mean_absolute_error: 1.7990
Epoch 45/100
303/303 [==============================] - 1s 2ms/step - loss: 7.1074 - mean_absolute_error: 1.7148
Epoch 46/100
303/303 [==============================] - 1s 2ms/step - loss: 7.5122 - mean_absolute_error: 1.7243
Epoch 47/100
303/303 [==============================] - 1s 2ms/step - loss: 7.9830 - mean_absolute_error: 1.7629
Epoch 48/100
303/303 [==============================] - 1s 2ms/step - loss: 7.9809 - mean_absolute_error: 1.7427
Epoch 49/100
303/303 [==============================] - 1s 2ms/step - loss: 7.5815 - mean_absolute_error: 1.7419
Epoch 50/100
303/303 [==============================] - 1s 2ms/step - loss: 7.4550 - mean_absolute_error: 1.7234
Epoch 51/100
303/303 [==============================] - 1s 2ms/step - loss: 7.6047 - mean_absolute_error: 1.7068
Epoch 52/100
303/303 [==============================] - 1s 2ms/step - loss: 7.2155 - mean_absolute_error: 1.7

303/303 [==============================] - 1s 2ms/step - loss: 8.3330 - mean_absolute_error: 2.1160
Epoch 16/100
303/303 [==============================] - 1s 2ms/step - loss: 8.6397 - mean_absolute_error: 2.0331
Epoch 17/100
303/303 [==============================] - 1s 2ms/step - loss: 8.2755 - mean_absolute_error: 2.0185
Epoch 18/100
303/303 [==============================] - 1s 2ms/step - loss: 8.4079 - mean_absolute_error: 1.9989
Epoch 19/100
303/303 [==============================] - 1s 2ms/step - loss: 7.6087 - mean_absolute_error: 1.9699
Epoch 20/100
303/303 [==============================] - 1s 2ms/step - loss: 8.0717 - mean_absolute_error: 2.0020
Epoch 21/100
303/303 [==============================] - 1s 2ms/step - loss: 7.9033 - mean_absolute_error: 1.9640
Epoch 22/100
303/303 [==============================] - 1s 2ms/step - loss: 7.8663 - mean_absolute_error: 1.9418
Epoch 23/100
303/303 [==============================] - 1s 2ms/step - loss: 7.5560 - mean_absolute_error: 1.9

303/303 [==============================] - 1s 2ms/step - loss: 4.3957 - mean_absolute_error: 1.4363
Epoch 88/100
303/303 [==============================] - 1s 3ms/step - loss: 4.1278 - mean_absolute_error: 1.4417
Epoch 89/100
303/303 [==============================] - 1s 3ms/step - loss: 4.1142 - mean_absolute_error: 1.3437
Epoch 90/100
303/303 [==============================] - 1s 2ms/step - loss: 4.4270 - mean_absolute_error: 1.4964
Epoch 91/100
303/303 [==============================] - 1s 2ms/step - loss: 4.3592 - mean_absolute_error: 1.4493
Epoch 92/100
303/303 [==============================] - 1s 2ms/step - loss: 3.7582 - mean_absolute_error: 1.3907
Epoch 93/100
303/303 [==============================] - 1s 3ms/step - loss: 3.8819 - mean_absolute_error: 1.3548
Epoch 94/100
303/303 [==============================] - 1s 2ms/step - loss: 3.8995 - mean_absolute_error: 1.4294
Epoch 95/100
303/303 [==============================] - 1s 2ms/step - loss: 4.0558 - mean_absolute_error: 1.4

303/303 [==============================] - 1s 2ms/step - loss: 6.2679 - mean_absolute_error: 1.5991
Epoch 59/100
303/303 [==============================] - 1s 2ms/step - loss: 5.9667 - mean_absolute_error: 1.5791
Epoch 60/100
303/303 [==============================] - 1s 2ms/step - loss: 6.1144 - mean_absolute_error: 1.5844
Epoch 61/100
303/303 [==============================] - 1s 2ms/step - loss: 5.9760 - mean_absolute_error: 1.5853
Epoch 62/100
303/303 [==============================] - 1s 3ms/step - loss: 5.6495 - mean_absolute_error: 1.5168
Epoch 63/100
303/303 [==============================] - 1s 2ms/step - loss: 5.7420 - mean_absolute_error: 1.5796
Epoch 64/100
303/303 [==============================] - 1s 2ms/step - loss: 5.5262 - mean_absolute_error: 1.5320
Epoch 65/100
303/303 [==============================] - 1s 3ms/step - loss: 5.4305 - mean_absolute_error: 1.5126
Epoch 66/100
303/303 [==============================] - 1s 2ms/step - loss: 5.6549 - mean_absolute_error: 1.5

In [10]:
all_scores

[2.3428894600065626, 2.163103396349614, 2.955633845659766, 2.207315009419281]

In [11]:
np.mean(all_scores)

2.417235427858806

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 64)                896       
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________
